## Prevendo o preço de casas

Base de dados simplicada do desafio [House Prices](https://www.kaggle.com/c/house-prices-advanced-regression-techniques) do Kaggle.

Vamos tentar prever o preço de casas baseado no seu tamanho, ano de construção e capacidade de carros na garagem.

Durante a modelagem, poderemos observar como o MLFlow vai nos ajudar a acompanhar os resultados do experimento e tornar o nosso projeto mais maduro.

In [42]:
import pandas as pd

In [43]:
df = pd.read_csv('casas.csv')
df.head()

,tamanho,ano,garagem,preco
0,159.0,2003,2,208500
1,117.0,1976,2,181500
2,166.0,2001,2,223500
3,160.0,1915,3,140000
4,204.0,2000,3,250000


In [44]:
X = df.drop('preco', axis=1)
y = df['preco']

## Treino e teste

In [45]:
from sklearn.model_selection import train_test_split

In [46]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

In [47]:
X_train.head()

,tamanho,ano,garagem
135,156.0,1970,2
1452,100.0,2005,2
762,144.0,2009,2
932,177.0,2006,3
435,154.0,1996,2


## Regressão Linear

Primeiro vamos ajustar uma regressão linear ao nosso modelo. Lembrando que vamos usar a regressão linear e outras técnicas de forma bem simples, uma vez que o objetivo do nosso experimento não é uma ótima performance preditiva mas sim como o MLFlow pode ser útil para a fase de experimentação.

In [48]:
from sklearn.linear_model import LinearRegression

In [69]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [50]:
lr.predict([[120,2001,2]])

array([191436.53503831])

In [51]:
from sklearn.metrics import mean_squared_error, r2_score

In [52]:
lr_predicted = lr.predict(X_test)

In [53]:
import math

mse = mean_squared_error(y_test,lr_predicted)
rmse = math.sqrt(mse)
r2 = r2_score(y_test,lr_predicted)

In [54]:
print(f'mse: {mse}, rmse: {rmse}, r2: {r2}')

mse: 2078666917.9289908, rmse: 45592.39978251848, r2: 0.7021153642898048


## XGBoost

Uma abordagem comum durante o processo de experimentação é testar outras técnicas e também fazer a tunagem dos parâmetros. Para ver se o nosso modelo pode ser melhorado, vamos testar o ajuste de um modelo com o XGBoost.

In [70]:
from xgboost import XGBRegressor

In [71]:
xgb = XGBRegressor(random_state=42)
xgb.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1,
             objective='reg:squarederror', random_state=42, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [72]:
xgb_predicted = xgb.predict(X_test)
mse = mean_squared_error(y_test,xgb_predicted)
rmse = math.sqrt(mse)
r2 = r2_score(y_test,xgb_predicted)

In [73]:
print(f'mse: {mse}, rmse: {rmse}, r2: {r2}')

mse: 1550816053.129738, rmse: 39380.40189141977, r2: 0.7777593557411607


### Testando novos parâmetros

In [74]:
xgb = XGBRegressor(learning_rate=0.2,n_estimators=50,random_state=42)
xgb.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.2, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=50, n_jobs=8, num_parallel_tree=1,
             objective='reg:squarederror', random_state=42, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [75]:
xgb_predicted = xgb.predict(X_test)
mse = mean_squared_error(y_test,xgb_predicted)
rmse = math.sqrt(mse)
r2 = r2_score(y_test,xgb_predicted)
print(f'mse: {mse}, rmse: {rmse}, r2: {r2}')

mse: 1386727460.1346002, rmse: 37238.789724353286, r2: 0.8012741720529797


## Usando o MLFlow para fazer o tracking dos experimentos

In [76]:
import mlflow

### Regressão Linear

In [77]:
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_predicted = lr.predict(X_test)

In [78]:
mse = mean_squared_error(y_test,lr_predicted)
rmse = math.sqrt(mse)
r2 = r2_score(y_test,lr_predicted)
mlflow.log_metric("mse", mse)
mlflow.log_metric("rmse", rmse)
mlflow.log_metric("r2", r2)

In [79]:
mlflow.sklearn.log_model(lr, "lr")

In [80]:
print(f'mse: {mse}, rmse: {rmse}, r2: {r2}')

mse: 2078666917.9289908, rmse: 45592.39978251848, r2: 0.7021153642898048


### XGBoost

In [85]:
xgb_params = {
    'learning_rate':0.2,
    'n_estimators':50,
    'random_state':42
}
xgb = XGBRegressor(**xgb_params)
xgb.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.2, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=50, n_jobs=8, num_parallel_tree=1,
             objective='reg:squarederror', random_state=42, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [86]:
xgb_predicted = xgb.predict(X_test)
mse = mean_squared_error(y_test,xgb_predicted)
rmse = math.sqrt(mse)
r2 = r2_score(y_test,xgb_predicted)
print(f'mse: {mse}, rmse: {rmse}, r2: {r2}')

mse: 1386727460.1346002, rmse: 37238.789724353286, r2: 0.8012741720529797


In [84]:
mlflow.log_metric("mse", mse)
mlflow.log_metric("rmse", rmse)
mlflow.log_metric("r2", r2)

In [87]:
mlflow.xgboost.log_model(xgb, "xgboost")